In [1]:
import requests
import pickle
import os
import pandas as pd
import numpy as np
import datetime as dt
from api_connector import API_Connector
import praw
import pickle
from reddit_api import Reddit_API
api_keys = pickle.load(open('./monkey.pkl','rb'))
praw_reddit = praw.Reddit(client_id=api_keys['personal use script'],
                     client_secret=api_keys['Token'],
                     user_agent= api_keys['app name'],
                     username=api_keys['username'],
                     password= api_keys['password'])
reddit = Reddit_API()
reddit.connect_reddit(pickle_name = 'monkey')

<Response [200]>


## Scrapping Subreddits

i.e. is it nsfw, how many subscribers etc...

In [4]:
reddit.res

<Response [200]>

### Searching for subreddits

In [16]:
where = "popular"
# other options: 'new','gold',default'
res = requests.get(f"https://oauth.reddit.com/subreddits/{where}",params = params,
                   headers=reddit.headers)

In [20]:
res.json()['data']['children'][0]

features = ['subscribers','description','created','subreddit_type',
            'over18']

### Subreddit

In [32]:
# about
subreddit = "datascience"
res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/about",params = params,
                   headers=reddit.headers)
features = ['created','display_name','title','accounts_active','public_description', 'restrict_posting',
          'subscribers', 'quarantine', 'comment_score_hide_mins',  'accounts_active_is_fuzzed',
            'over18']
data = {subreddit: res.json()['data']}
pd.DataFrame(data).T[features]

,created,display_name,title,accounts_active,public_description,restrict_posting,subscribers,quarantine,comment_score_hide_mins,accounts_active_is_fuzzed,over18
datascience,1312631022.0,datascience,Data Science,666,A place for data science practitioners and pro...,True,909208,False,30,False,False


In [51]:
# rising
subreddit = "datascience"
params = {'limit': 100}
res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/rising",params = params,
                   headers=reddit.headers)
data = {}
for i, post in enumerate(res.json()['data']['children']):
    data[i] = post['data']
rising_posts = pd.DataFrame(data).T
print(rising_posts.shape)
rising_posts.head(2)

(25, 113)


,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,created_utc,num_crossposts,media,is_video,media_metadata,post_hint,url_overridden_by_dest,preview,crosspost_parent_list,crosspost_parent
0,None,datascience,i used xgboost for my unbalanced data after ba...,t2_a8ditcldc,False,None,0,False,adding mahalanobis distance as a feature,[],...,1685045472.0,0,None,False,NaN,NaN,NaN,NaN,NaN,NaN
1,None,datascience,I’m wondering why there hasn’t been more adopt...,t2_i69qgpqa,False,None,0,False,Why hasn’t there been a shift from tableau to ...,[],...,1685023681.0,0,None,False,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
list(rising_posts.columns)

['approved_at_utc',
 'subreddit',
 'selftext',
 'author_fullname',
 'saved',
 'mod_reason_title',
 'gilded',
 'clicked',
 'title',
 'link_flair_richtext',
 'subreddit_name_prefixed',
 'hidden',
 'pwls',
 'link_flair_css_class',
 'downs',
 'thumbnail_height',
 'top_awarded_type',
 'hide_score',
 'name',
 'quarantine',
 'link_flair_text_color',
 'upvote_ratio',
 'author_flair_background_color',
 'subreddit_type',
 'ups',
 'total_awards_received',
 'media_embed',
 'thumbnail_width',
 'author_flair_template_id',
 'is_original_content',
 'user_reports',
 'secure_media',
 'is_reddit_media_domain',
 'is_meta',
 'category',
 'secure_media_embed',
 'link_flair_text',
 'can_mod_post',
 'score',
 'approved_by',
 'is_created_from_ads_ui',
 'author_premium',
 'thumbnail',
 'edited',
 'author_flair_css_class',
 'author_flair_richtext',
 'gildings',
 'content_categories',
 'is_self',
 'mod_note',
 'created',
 'link_flair_type',
 'wls',
 'removed_by_category',
 'banned_by',
 'author_flair_type',
 'dom

In [85]:
rising_posts.author.unique()

array(['qhelspil', 'AdFew4357', 'NDVGuy', 'Beneficial_Space9001',
       'Inquation', 'Environmental-Bet-37', 'Gonduska', 'Andy-VertaAI',
       'blitzarx1', 'LordStaccato', 'Calligraphiti', 'Data_cruncher',
       'Outrageous_Hawk4549', 'Accomplished-Clock56', 'DidItForTheData',
       'Embarrassed-Piano825', 'LatterConcentrate6', 'chacalgamer',
       'Bayesian8', 'No-Wallaby5033', 'Long_Explorer_2731', 'data-leon',
       'madhav1113', 'Bitter-Tell-8088'], dtype=object)

### User

In [172]:
# about  
user = 'chacalgamer'
res = requests.get(f"https://oauth.reddit.com/u/{user}/about",
                   headers=reddit.headers)
features = ['created','id','verified','is_gold', 'is_mod','comment_karma','total_karma',
           ] 
data_ = {}
data = {}
for i,  feature in enumerate(features):
    data_[feature] = res.json()['data'][feature]
data[user] = data_
pd.DataFrame(data).T


,comment_karma,created,id,is_gold,is_mod,total_karma,verified
chacalgamer,4574,1433381032.0,nw83t,False,False,5404,True


In [134]:
# comments
user = 'DidItForTheData'
res = requests.get(f"https://oauth.reddit.com/user/{user}/comments",params = {'limit': 100},
                   headers=reddit.headers)
data = {}
for i, post in enumerate(res.json()['data']['children']):
    data[i] = post['data']
user_comments = pd.DataFrame(data).T
user_comments.head()

,subreddit_id,approved_at_utc,author_is_blocked,comment_type,link_title,mod_reason_by,banned_by,ups,num_reports,author_flair_type,...,awarders,all_awardings,locked,author_flair_background_color,collapsed_because_crowd_control,mod_reports,quarantine,mod_note,link_url,media_metadata
0,t5_2rvat,None,False,None,Is this good for taking bike videos for YouTube?,None,None,3,None,text,...,[],[],False,None,None,[],False,None,https://i.redd.it/ge9c1zkvd22b1.jpg,NaN
1,t5_2rvat,None,False,None,Is this good for taking bike videos for YouTube?,None,None,10,None,text,...,[],[],False,None,None,[],False,None,https://i.redd.it/ge9c1zkvd22b1.jpg,NaN
2,t5_4jbzqe,None,False,None,When you preorder a truck does the dealer have...,None,None,1,None,text,...,[],[],False,None,None,[],False,None,https://www.reddit.com/r/FordMaverickTruck/com...,NaN
3,t5_4jbzqe,None,False,None,When you preorder a truck does the dealer have...,None,None,7,None,text,...,[],[],False,None,None,[],False,None,https://www.reddit.com/r/FordMaverickTruck/com...,NaN
4,t5_4jbzqe,None,False,None,When you preorder a truck does the dealer have...,None,None,18,None,text,...,[],[],False,None,None,[],False,None,https://www.reddit.com/r/FordMaverickTruck/com...,NaN


In [135]:
user_comments.columns

Index(['subreddit_id', 'approved_at_utc', 'author_is_blocked', 'comment_type',
       'link_title', 'mod_reason_by', 'banned_by', 'ups', 'num_reports',
       'author_flair_type', 'total_awards_received', 'subreddit',
       'link_author', 'likes', 'replies', 'user_reports', 'saved', 'id',
       'banned_at_utc', 'mod_reason_title', 'gilded', 'archived',
       'collapsed_reason_code', 'no_follow', 'author', 'num_comments',
       'can_mod_post', 'send_replies', 'parent_id', 'score', 'author_fullname',
       'over_18', 'report_reasons', 'removal_reason', 'approved_by',
       'controversiality', 'body', 'edited', 'top_awarded_type', 'downs',
       'author_flair_css_class', 'is_submitter', 'collapsed',
       'author_flair_richtext', 'author_patreon_flair', 'body_html',
       'gildings', 'collapsed_reason', 'distinguished', 'associated_award',
       'stickied', 'author_premium', 'can_gild', 'link_id',
       'unrepliable_reason', 'author_flair_text_color', 'score_hidden',
       'pe

In [105]:
for post in res.json()['data']['children']:
    print(post['kind'])#[0]['data']

KeyError: 'children'

In [103]:
reddit.user_comment_pull('Outrageous_Hawk4549')

,created_utc,created_cst,subreddit,ups,kind,id,body,comment_length
0,2023-05-25 20:41:38,2023-05-25 15:41:38,datascience,1.0,t1,jlm3nyl,It is paid,10.0
1,2023-05-17 12:55:40,2023-05-17 07:55:40,u_Outrageous_Hawk4549,4.0,t1,jkhslsx,Have you ever been in a situation where you ha...,85.0
2,2023-05-17 12:52:41,2023-05-17 07:52:41,u_Outrageous_Hawk4549,4.0,t1,jkhs88g,Have you ever been starstruck by someone you n...,81.0
3,2023-05-17 12:49:20,2023-05-17 07:49:20,u_Outrageous_Hawk4549,5.0,t1,jkhrt8p,What's the most ridiculous thing you've ever d...,91.0
4,2023-05-17 12:46:07,2023-05-17 07:46:07,u_Outrageous_Hawk4549,5.0,t1,jkhretb,"If you could have any job for just one day, wh...",69.0
5,2023-05-17 12:42:56,2023-05-17 07:42:56,u_Outrageous_Hawk4549,4.0,t1,jkhr0zh,"Have you ever made a spontaneous, impulsive de...",103.0
6,2023-05-17 12:39:40,2023-05-17 07:39:40,u_Outrageous_Hawk4549,4.0,t1,jkhqmpi,What is the most bizarre talent you have?,41.0
7,2023-05-17 12:36:24,2023-05-17 07:36:24,u_Outrageous_Hawk4549,4.0,t1,jkhq8bc,What’s the funniest prank you’ve ever pulled o...,56.0
8,2023-05-17 12:32:59,2023-05-17 07:32:59,u_Outrageous_Hawk4549,3.0,t1,jkhpto9,What's the most outrageous lie you've ever told?,48.0
9,2023-05-17 12:30:06,2023-05-17 07:30:06,u_Outrageous_Hawk4549,3.0,t1,jkhphlm,What is the most important lesson you have lea...,66.0
